This notebook scrapes 2022-2023 season data weekly and outputs it as a csv file, and a txt file documenting the date and number of games.

In [2]:
# Import packages
from bs4 import BeautifulSoup
import requests
import pandas as pd
import warnings
warnings.filterwarnings('ignore')
from datetime import datetime
import numpy as np
from IPython.display import clear_output
from time import sleep

# Set directory path
path = '/Users/martinbogaert/Desktop/NBA Data Analysis/nba_ai_awards/'

print('MAKE SURE THAT YOU CHANGE THE WEEK VARIABLE BELOW')

MAKE SURE THAT YOU CHANGE THE WEEK VARIABLE BELOW


In [3]:
scrape_year = 2023 # change to 2023
# Date and time of scrapping
week = 25 # CHANGE THIS
date1 = datetime.now().strftime('%d/%m')
date2 = datetime.now().strftime('%d %B %Y')
time = datetime.now().strftime('%H:%M:%S')
print(date2 + ', ' + time)

11 April 2023, 13:26:43


In [4]:
# Define function which deals with players having played on multiple teams
def single_player(df):
    if len(df) == 1: # if only one team, keep it
        return df
    else: # multiple teams: keep total (TOT) stats and replace team with last played for
        row = df[df['Tm'] == 'TOT'].copy()
        row['Tm'] = str(df['Tm'].iloc[-1])
        return row
        
# Team abbreviation dictionary
team_dic = {'Sacramento Kings': 'SAC', 'Los Angeles Clippers': 'LAC','Washington Wizards': 'WAS','Phoenix Suns': 'PHO',
   'Boston Celtics': 'BOS','Golden State Warriors': 'GSW','Denver Nuggets': 'DEN','Orlando Magic': 'ORL','Chicago Bulls': 'CHI',
   'Utah Jazz': 'UTA','Toronto Raptors': 'TOR','Los Angeles Lakers': 'LAL','Portland Trail Blazers': 'POR',
   'Memphis Grizzlies': 'MEM','Miami Heat': 'MIA','Atlanta Hawks': 'ATL','Oklahoma City Thunder': 'OKC','Milwaukee Bucks': 'MIL',
   'San Antonio Spurs': 'SAS','Charlotte Hornets': 'CHO','Brooklyn Nets': 'BRK','Cleveland Cavaliers': 'CLE','Philadelphia 76ers': 'PHI',
   'Houston Rockets': 'HOU','Dallas Mavericks': 'DAL','New York Knicks': 'NYK','Indiana Pacers': 'IND','Detroit Pistons': 'DET',
   'Minnesota Timberwolves': 'MIN','New Orleans Pelicans': 'NOP','Charlotte Hornets\xa0': 'CHO'
           }


# Player stats

In [5]:
# Scrape player per game and advanced stats

dfs = [] 
for Type in ['per_game', 'advanced']: # Loop twice: per game, advanced stats

    # Scrape webpage table
    url = f'https://www.basketball-reference.com/leagues/NBA_{scrape_year}_{Type}.html' 
    page = requests.get(url)
    soup = BeautifulSoup(page.content, 'html')
    table = soup.find('table')
    # Transform table into DataFrame
    stats = pd.read_html(str(table))[0]
    stats = stats[stats['Rk'] != 'Rk'].reset_index(drop = True)
    del stats['Rk'] # Drop rank column

    dfs.append(stats) # Store both DataFrames

# Find unique columns, deal with repeating columns
col_pg = list(dfs[0]) 
col_pg.remove('Player'); col_pg.remove('Tm') # Remove merge columns (Player & team)
temp = list(dfs[1])
unique_cols = [x for x in temp if x not in col_pg] # Keep advanced columns if they are not already in per game columns

# Merge per game and advanced data
data = pd.merge(dfs[0], dfs[1][unique_cols], on = ['Player', 'Tm'])
data['Year'] = len(data) * [scrape_year] # Add year
del data['Unnamed: 19']
del data['Unnamed: 24']

data['Player'] = data['Player'].str.replace('*', '', regex = False) # Delete * in some names
data = data.groupby(['Player']).apply(single_player) # Keep one entry for multiple teams players
data = data.reset_index(drop = True)
data = data.apply(pd.to_numeric, errors = 'ignore')
# Calculate projected win shares
for stat in ['OWS', 'DWS', 'WS', 'VORP'] :
    data[stat] = data[stat] + data[stat] / data['G'] * (82 - data['G'])

data.head()

,Player,Pos,Age,Tm,G,GS,MP,FG,FGA,FG%,...,USG%,OWS,DWS,WS,WS/48,OBPM,DBPM,BPM,VORP,Year
0,A.J. Green,SG,23,MIL,35,1,9.9,1.5,3.6,0.424,...,16.6,1.171429,0.702857,1.874286,0.111,-0.3,-0.6,-0.9,0.234286,2023
1,A.J. Lawson,SG,22,DAL,15,0,7.2,1.5,2.9,0.500,...,21.1,0.546667,0.546667,0.546667,0.063,-1.5,-2.9,-4.4,-0.546667,2023
2,AJ Griffin,SF,19,ATL,72,12,19.5,3.4,7.4,0.465,...,17.9,1.480556,1.025000,2.505556,0.075,-0.3,-0.5,-0.8,0.455556,2023
3,Aaron Gordon,PF,27,DEN,68,68,30.2,6.3,11.2,0.564,...,21.1,5.547059,2.652941,8.200000,0.158,2.4,-0.3,2.1,2.532353,2023
4,Aaron Holiday,PG,26,ATL,63,6,13.4,1.5,3.5,0.418,...,13.4,0.390476,0.780952,1.171429,0.053,-2.9,0.9,-2.0,0.000000,2023


In [6]:
# Srape individual ORtg and DRtg (can only be found in team data)
dfs_rtg = []
for i, tm in enumerate(set(data['Tm'])) :
    
    prog = round(100 * (i+1) / len(set(data['Tm'])), 2) # Compute and print loop progress
    print(f'{prog}% {tm} ...')
    clear_output(wait = True)
    
    # Scrape webpage table
    url = f'https://www.basketball-reference.com/teams/{tm}/{scrape_year}.html'
    page = requests.get(url)
    table = page.text.split(f'id="div_per_poss">')[1].split('</table>')[0] + '</table>'
    soup = BeautifulSoup(table, 'html')
    # Transform table into DataFrame
    data_rtg = pd.read_html(str(soup))[0]
    data_rtg['Tm'] = len(data_rtg) * [tm] # Add team and year
    data_rtg['Year'] = len(data_rtg) * [scrape_year]

    data_rtg = data_rtg.rename(columns = {'Unnamed: 1' : 'Player'})[['Player','Tm','Year','ORtg','DRtg']]
    dfs_rtg.append(data_rtg)

    sleep(2)

# Merge data with individual ORtg and DRtg
data_rtg = pd.concat(dfs_rtg).rename(columns = {'ORtg':'ORtg/100', 'DRtg':'DRtg/100'})
data_rtg['Player'] = data_rtg['Player'].str.replace('*', '', regex = False) # Clean player name column
data = data.merge(data_rtg, on = ['Player','Year','Tm'], how = 'inner') # Merge ratings with data
data.head()

,Player,Pos,Age,Tm,G,GS,MP,FG,FGA,FG%,...,DWS,WS,WS/48,OBPM,DBPM,BPM,VORP,Year,ORtg/100,DRtg/100
0,A.J. Green,SG,23,MIL,35,1,9.9,1.5,3.6,0.424,...,0.702857,1.874286,0.111,-0.3,-0.6,-0.9,0.234286,2023,121.0,115
1,A.J. Lawson,SG,22,DAL,15,0,7.2,1.5,2.9,0.500,...,0.546667,0.546667,0.063,-1.5,-2.9,-4.4,-0.546667,2023,111.0,119
2,AJ Griffin,SF,19,ATL,72,12,19.5,3.4,7.4,0.465,...,1.025000,2.505556,0.075,-0.3,-0.5,-0.8,0.455556,2023,116.0,119
3,Aaron Gordon,PF,27,DEN,68,68,30.2,6.3,11.2,0.564,...,2.652941,8.200000,0.158,2.4,-0.3,2.1,2.532353,2023,124.0,115
4,Aaron Holiday,PG,26,ATL,63,6,13.4,1.5,3.5,0.418,...,0.780952,1.171429,0.053,-2.9,0.9,-2.0,0.000000,2023,110.0,117


# Team stats

In [7]:
# Scrape team stats
url = f'https://www.basketball-reference.com/leagues/NBA_{scrape_year}.html'
page = requests.get(url)

soup = BeautifulSoup(page.content, 'html')

while soup.find('tr', class_ = 'thead') is not None:
    soup.find('tr', class_ = 'thead').decompose()

table_E = soup.find('table', id = 'divs_standings_E')
teams_E = pd.read_html(str(table_E))[0]
teams_E = teams_E.rename(columns = {'Eastern Conference': 'Team'})

table_W = soup.find('table', id = 'divs_standings_W')
teams_W = pd.read_html(str(table_W))[0]
teams_W = teams_W.rename(columns = {'Western Conference': 'Team'})

teams_E['Seed'] = [len(teams_E[teams_E['W/L%'] > wl])+1 for wl in teams_E['W/L%']]
teams_W['Seed'] = [len(teams_W[teams_W['W/L%'] > wl])+1 for wl in teams_W['W/L%']]

teams = pd.concat([teams_E, teams_W])

del teams['GB']
teams['Team'] = teams['Team'].str.replace('*','', regex = False)
teams['Team'] = teams['Team'].apply(lambda x: ' '.join(x.split()[:-1]))

url = f'https://www.basketball-reference.com/leagues/NBA_{scrape_year}_ratings.html'
page = requests.get(url)
soup = BeautifulSoup(page.content, 'html')
soup.find('tr', class_='over_header').decompose()    
table = soup.find('table')
ratings = pd.read_html(str(table))[0][['Team', 'MOV', 'ORtg', 'DRtg', 'NRtg', 'MOV/A', 'ORtg/A', 'DRtg/A', 'NRtg/A']]
ratings['Team'] = ratings['Team'].str.replace('Charlotte Hornets\xa0', 'Charlotte Hornets')
teams['Team'] = teams['Team'].str.replace('Charlotte Hornets\xa0', 'Charlotte Hornets')

teams = teams.merge(ratings, on = 'Team', how = 'outer')
teams = teams.sort_values('W/L%', ascending = False).reset_index(drop = True)
teams['Tm'] = [team_dic.get(tm) for tm in teams['Team']]

# Merge indiviudal and team stats
stats = data.merge(teams, how = 'inner', on = 'Tm')
stats.head()

,Player,Pos,Age,Tm,G,GS,MP,FG,FGA,FG%,...,SRS,Seed,MOV,ORtg,DRtg,NRtg,MOV/A,ORtg/A,DRtg/A,NRtg/A
0,A.J. Green,SG,23,MIL,35,1,9.9,1.5,3.6,0.424,...,3.61,1,3.63,116.13,112.64,3.49,3.61,116.12,112.64,3.48
1,Bobby Portis,PF,27,MIL,70,22,26.0,5.7,11.5,0.496,...,3.61,1,3.63,116.13,112.64,3.49,3.61,116.12,112.64,3.48
2,Brook Lopez,C,34,MIL,78,78,30.4,6.1,11.5,0.531,...,3.61,1,3.63,116.13,112.64,3.49,3.61,116.12,112.64,3.48
3,Giannis Antetokounmpo,PF,28,MIL,63,63,32.1,11.2,20.3,0.553,...,3.61,1,3.63,116.13,112.64,3.49,3.61,116.12,112.64,3.48
4,Goran Dragić,PG,36,MIL,58,0,15.0,2.4,5.8,0.421,...,3.61,1,3.63,116.13,112.64,3.49,3.61,116.12,112.64,3.48


In [8]:
# Determine players eligible for ROY and SMOY
url = f'https://www.basketball-reference.com/leagues/NBA_{scrape_year}_rookies.html' # Scrape rookie list
page = requests.get(url)
soup = BeautifulSoup(page.content, 'html')

for overhead in ['over_header', 'over_header thead', 'thead']:
    while soup.find('tr', class_ = overhead) is not None:
        soup.find('tr', class_ = overhead).decompose()
        
table = soup.find('table')
temp = pd.read_html(str(table))[0]
rookies = list(temp['Player'])

stats['roy'] = [1 if player in rookies else 0 for player in stats['Player']] # flag eligible rookies
stats['smoy'] = [1 if gs/g <= 0.5 else 0 for gs, g in zip(stats['GS'], stats['G'])] # flag eligible sixth-mans

for stat in list(stats):
    stats[stat] = stats[stat].replace(np.nan, 0) # replace NaNs with zeros



stats.sort_values('PTS', ascending = False).head()

,Player,Pos,Age,Tm,G,GS,MP,FG,FGA,FG%,...,MOV,ORtg,DRtg,NRtg,MOV/A,ORtg/A,DRtg/A,NRtg/A,roy,smoy
514,Joel Embiid,C,28,PHI,66,66,34.6,11.0,20.1,0.548,...,4.32,118.87,114.51,4.35,4.37,118.98,114.56,4.42,0,0
31,Luka Dončić,PG,23,DAL,66,66,36.2,10.9,22.0,0.496,...,0.07,117.63,117.56,0.06,-0.14,117.54,117.70,-0.16,0,0
291,Damian Lillard,PG,32,POR,58,58,36.3,9.6,20.7,0.463,...,-4.01,115.72,119.83,-4.11,-3.96,115.72,119.79,-4.07,0,0
105,Shai Gilgeous-Alexander,PG,24,OKC,68,68,35.5,10.4,20.3,0.510,...,1.09,115.92,114.90,1.03,0.96,115.80,114.91,0.89,0,0
3,Giannis Antetokounmpo,PF,28,MIL,63,63,32.1,11.2,20.3,0.553,...,3.63,116.13,112.64,3.49,3.61,116.12,112.64,3.48,0,0


In [9]:
# Export data to csv
stats.to_csv(path + f'Algorithm/weekly data/week_{week}.csv', index = None)

# Export weekly info to txt
n = teams['W'].sum()
with open(path + f'Algorithm/weekly data/week_{week}.txt', 'w+') as file:
    file.write(f'Date : {date1}\n')
    file.write(f'Date : {date2}\n')
    file.write(f'No. games : {n}\n')
    file.write(f'Time : {time}\n')
    file.close()

In [10]:
print(stats.shape)

(538, 70)
